In [ ]:
import re
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import pickle
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('Shakespeare_data.csv')
print(data.head())

   Dataline      Play  PlayerLinenumber ActSceneLine         Player  \
0         1  Henry IV               NaN          NaN            NaN   
1         2  Henry IV               NaN          NaN            NaN   
2         3  Henry IV               NaN          NaN            NaN   
3         4  Henry IV               1.0        1.1.1  KING HENRY IV   
4         5  Henry IV               1.0        1.1.2  KING HENRY IV   

                                          PlayerLine  
0                                              ACT I  
1                       SCENE I. London. The palace.  
2  Enter KING HENRY, LORD JOHN OF LANCASTER, the ...  
3             So shaken as we are, so wan with care,  
4         Find we a time for frighted peace to pant,  


In [ ]:
# getting text from the data
text = []
for i in data['PlayerLine']:
    text.append(i)

# lets see how the text is looking
text[:5]

['ACT I',
 'SCENE I. London. The palace.',
 'Enter KING HENRY, LORD JOHN OF LANCASTER, the EARL of WESTMORELAND, SIR WALTER BLUNT, and others',
 'So shaken as we are, so wan with care,',
 'Find we a time for frighted peace to pant,']

In [ ]:
# Text Cleaning
def clean_text(text):
    # removing special characters like @, #, $, etc
    pattern = re.compile('[^a-zA-z0-9\s]')
    text = re.sub(pattern,'',text)

    # removing digits
    pattern = re.compile('\d+')
    text = re.sub(pattern,'',text)

    # converting text to lower case
    text = text.lower()
    return text

texts = []
for t in text:
    new_text = clean_text(t)
    texts.append(new_text)

# cleaned text
texts[:5]

['act i',
 'scene i london the palace',
 'enter king henry lord john of lancaster the earl of westmoreland sir walter blunt and others',
 'so shaken as we are so wan with care',
 'find we a time for frighted peace to pant']

In [ ]:
# lets take first 10000 words for the model training
texts = texts[:10000]

# using tensorflow tokenizer and
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# generating text sequences, i.e. encoding the text
text_sequences = tokenizer.texts_to_sequences(texts) # Remove np.array()
print('Text -->>',texts[0])
print('Embedding -->>',text_sequences[0])

# padding the sequences
Max_Sequence_Len = max([len(x) for x in text_sequences])
text_sequences = pad_sequences(text_sequences,
                               maxlen = Max_Sequence_Len, padding='pre') # pad_sequences expects a list

print('Maximum Sequence Length -->>',Max_Sequence_Len)
print('Text Sequence -->>\n',text_sequences[0])
print('Text Sequence Shape -->>',text_sequences.shape)

Text -->> act i
Embedding -->> [455, 4]
Maximum Sequence Length -->> 54
Text Sequence -->>
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 455   4]
Text Sequence Shape -->> (10000, 54)


In [ ]:
# getting X and y from the data
X, y = text_sequences[:, :-1], text_sequences[:,-1]
print('First Input :',X[0])
print('First Target :',y[0])

word_index = tokenizer.word_index

# using one hot encoding on y
total_words = len(word_index) + 1
print('Total Number of Words:',total_words)

y = to_categorical(y, num_classes=total_words)

# printing X and y shapes
print('Input Shape :',X.shape)
print('Target Shape :',y.shape)

First Input : [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 455]
First Target : 4
Total Number of Words: 7865
Input Shape : (10000, 53)
Target Shape : (10000, 7865)


In [ ]:
model = Sequential(name="LSTM_Model")

# adding embedding layer
model.add(Embedding(total_words,
                    Max_Sequence_Len-1,
                    input_length=Max_Sequence_Len-1))

# adding a LSTM layer
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.5))

#adding the final output activation with activation function of softmax
model.add(Dense(total_words, activation='softmax'))

# printing model summary
print(model.summary())

Model: "LSTM_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
# Compiling the model
model.compile(
    loss="categorical_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

# Training the LSTM model
history = model.fit(X, y,
                       epochs=50,
                       verbose=1)

Epoch 1/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 216s 677ms/step - accuracy: 0.0107 - loss: 8.2064
Epoch 2/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 257s 660ms/step - accuracy: 0.0196 - loss: 7.3656
Epoch 3/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 206s 660ms/step - accuracy: 0.0270 - loss: 7.0830
Epoch 4/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 263s 662ms/step - accuracy: 0.0276 - loss: 6.8596
Epoch 5/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 259s 653ms/step - accuracy: 0.0311 - loss: 6.5556
Epoch 6/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 262s 654ms/step - accuracy: 0.0384 - loss: 6.2501
Epoch 7/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 203s 649ms/step - accuracy: 0.0473 - loss: 5.8923
Epoch 8/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 267s 664ms/step - accuracy: 0.0657 - loss: 5.4536
Epoch 9/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 263s 668ms/step - accuracy: 0.0972 - loss: 5.0058
Epoch 10/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 210s 671ms/step - accuracy: 0.1351 - loss: 4.5055
Epoch 11/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 262s 670ms/step - accuracy: 0.1954 - loss: 4.0026
Epoch 12

In [ ]:
def autoCompletations(text, model):
    # Tokenization and Text vectorization
    text_sequences = np.array(tokenizer.texts_to_sequences([text]))
    # Pre-padding
    testing = pad_sequences(text_sequences, maxlen = Max_Sequence_Len-1, padding='pre')
    # Prediction
    y_pred_test = np.argmax(model.predict(testing,verbose=0))

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
        if index == y_pred_test:
            predicted_word = word
            break
    text += " " + predicted_word + '.'
    return text

complete_sentence = autoCompletations('I have seen this', model)
complete_sentence

'I have seen this cause.'

In [ ]:
def generate_text(text, new_words):
    for _ in range(new_words):
        text = autoCompletations(text, model)[:-1]
    return text

generated_text = generate_text('I have seen', 5)
generated_text

'I have seen him now on me now'

In [ ]:
# saving the model
model.save('sentence_completion.h5')

# saving the tokenizer
filename = 'tokenizer.pkl'
pickle.dump(tokenizer, open(filename, 'wb'))